In [1]:
import numpy as np
import pandas as pd
#import numpy and pandas libraries

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import matplotlib.pyplot as plt

#import necessary libraries in Pytorch and matplotlib

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#we import MinMaxScaler function from sklearn to scale our dataset and also make a instance of that

In [18]:
def format_train_test (batch_size,data,percent,num_predicts):
    length_train = int(data.shape[0]*(1-percent))
    a1 = length_train//batch_size
    length_train = batch_size*a1
    train_data = data.iloc[:,1:2].values
    train_data = scaler.fit_transform(train_data)
    train_set = []
    train_label = []
    for i in range(length_train):
        train_set.append(train_data[i:i + num_predicts])
        train_label.append(train_data[i + num_predicts:i + 2*num_predicts])
    
    length_test = int(data.shape[0] - length_train - 4*num_predicts)
    a2 = length_test//batch_size
    length_test = int(batch_size*a2) 
    test = data.iloc[:,1:2].values
    test = scaler.fit_transform(test)
    test_data = test[length_train + 2*num_predicts:length_train + length_test + 4*num_predicts]
    test_set = []
    test_label = []
    for j in range(length_test):
        test_set.append(test_data[j:j + num_predicts])
        test_label.append(test_data[j + num_predicts:j + 2*num_predicts])
        
    
    train_set = np.reshape(np.array(train_set),(np.array(train_set).shape[0], np.array(train_set).shape[1], 1))
    train_label = np.reshape(np.array(train_label),(np.array(train_label).shape[0], np.array(train_label).shape[1]))
    test_set = np.reshape(np.array(test_set),(np.array(test_set).shape[0], np.array(test_set).shape[1], 1))
    test_label = np.reshape(np.array(test_label),(np.array(test_label).shape[0], np.array(test_label).shape[1]))
    
    return train_set, train_label, test_set, test_label
#Here is the function to make train set, test set, and label for each of them from the dataframe. We first want to see how many
#We want to find the percent of data used for train set. Then, we find length of train data so that it could divisible by batch 
#size. Then we scale the dataset and make train set and label for each by iterating through the dataset until and make them lag
#by number of predict steps we want until we have desired amount of data for train process. Then, we find the length for test
#set and we repeat the same process as forming train set and its label.

In [19]:
train_set, train_label, test_set, test_label = format_train_test(64,new_coin_df,0.2,5)

train_set, train_label, test_set, test_label = map(torch.tensor,(train_set, train_label, test_set, test_label))

train = TensorDataset(train_set, train_label)
train_tensor = DataLoader(train, batch_size = 64, shuffle=False)

test = TensorDataset(test_set, test_label)
test_tensor = DataLoader(test, batch_size= 64, shuffle=False)

#we then make train, test sets, and their labels. Then we convert them to torch tensors and make torch dataset and use DataLoader to 
#to make data ready for training and testing. Here batch size is 64, percent of test set is 0.2 and 5 predictions steps

In [20]:
class LSTM_model(nn.Module):
    def __init__(self, out_size, in_size, hidden_size1, hidden_size2,batch_size):
        super().__init__()
        self.batch_size = batch_size
        self.in_size = in_size
        self.out_size = out_size
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.lstm1 = nn.LSTM(input_size=self.in_size,hidden_size=self.hidden_size1,num_layers=2,batch_first=True)
        self.lstm2 = nn.LSTM(input_size=self.hidden_size1,hidden_size=self.hidden_size2,num_layers=2,batch_first=True)
        self.last = nn.Linear(self.hidden_size2, self.out_size)
    def forward(self,x,old_state):
        s1,state1 = self.lstm1(x,old_state)
        s2,state2 = self.lstm2(s1,state1)
        return self.last(s2), state2
    def initial_s(self, num_predicts):
        return (torch.zeros(2,self.batch_size, self.hidden_size1),
                torch.zeros(2,self.batch_size, self.hidden_size1))
#Here we make a class of LSTM models. We use two lstm steps with 2 layers each and then the dense layer at the end to output prediction
#Here we also need to keep track of the state for LSTM layers. We also make initial states with 2 zeros tensors

In [21]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
device
# we check for the availablibility of GPU to use for training

device(type='cuda', index=0)

In [22]:
model = LSTM_model(5,1,5,5,64)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
#we make model, upload it to GPU, make CrossEntropy loss instance as long as the Adam optimizer with learning rate of 0.001

In [23]:
print(model) #print the model summary

LSTM_model(
  (lstm1): LSTM(1, 5, num_layers=2, batch_first=True)
  (lstm2): LSTM(5, 5, num_layers=2, batch_first=True)
  (last): Linear(in_features=5, out_features=5, bias=True)
)


In [27]:
def train_test(model,train_tensor,test_tensor,criterion,optimizer,device,num_predicts,iterations):
    for i in range(iterations):
        h0, c0 = model.initial_s(num_predicts)
        h, c = h0.to(device), c0.to(device)
        for (x,y) in train_tensor:
            x, y = x.to(device), y.to(device)
            y_hat, (h, c) = model(x.float(), (h, c))
            loss = criterion(y_hat, y.long())
            optimizer.zero_grad()
            h = h.detach()
            c = c.detach()
            loss.backward()
            optimizer.step()
        with torch.no_grad():
            h01, c01 = model.initial_s(num_predicts)
            h1, c1 = h01.to(device), c01.to(device)
            lost_val = 0
            for (x1, y1) in test_tensor:
                x1, y1 = x1.to(device), y1.to(device)
                y_hat1, (h1, c1) = model(x1.float(),(h1,c1))
                loss1 = criterion(y_hat1, y1.long())
                lost_val += loss1.item()
                h1 = h1.detach()
                c1 = c1.detach()
            lost_val /= len(x1)
            print(f"Iteration {i+1}\n********************")
            print('The loss for train is: ',loss.item())
            print('The loss for test is: ',lost_val)
#Here we create function to do the training and validation process. We iterate through number of epochs, make initial state for model
#and iterate through each data batch to do prediction, compute the loss and do backward propagation to improve the parameters. In
#the same epoch, I also apply the model to make prediction on test dataset to keep track of model performace more closely. I also
#print iteration round, train loss, and test loss

In [28]:
train_test(model,train_tensor,test_tensor,criterion,optimizer,device,5,100)

Iteration 1
********************
The loss for train is:  1.580761194229126
The loss for test is:  0.2221520785242319
Iteration 2
********************
The loss for train is:  1.4932750463485718
The loss for test is:  0.20948922634124756
Iteration 3
********************
The loss for train is:  1.2867605686187744
The loss for test is:  0.17983133345842361
Iteration 4
********************
The loss for train is:  0.9603341221809387
The loss for test is:  0.1338758636265993
Iteration 5
********************
The loss for train is:  0.6447484493255615
The loss for test is:  0.09011944849044085
Iteration 6
********************
The loss for train is:  0.42319363355636597
The loss for test is:  0.05946787493303418
Iteration 7
********************
The loss for train is:  0.2900962829589844
The loss for test is:  0.04107495490461588
Iteration 8
********************
The loss for train is:  0.2100238800048828
The loss for test is:  0.03001663344912231
Iteration 9
********************
The loss for trai

Iteration 69
********************
The loss for train is:  0.0023341327905654907
The loss for test is:  0.0019883093518728856
Iteration 70
********************
The loss for train is:  0.0022670684847980738
The loss for test is:  0.001985856240935391
Iteration 71
********************
The loss for train is:  0.002202586503699422
The loss for test is:  0.0019836069222947117
Iteration 72
********************
The loss for train is:  0.0021408218890428543
The loss for test is:  0.0019816879103018437
Iteration 73
********************
The loss for train is:  0.0020813082810491323
The loss for test is:  0.0019798761459242087
Iteration 74
********************
The loss for train is:  0.0020231143571436405
The loss for test is:  0.0019785747717833146
Iteration 75
********************
The loss for train is:  0.0019691598135977983
The loss for test is:  0.001977394291316159
Iteration 76
********************
The loss for train is:  0.0019148109713569283
The loss for test is:  0.001976249657673179
Iter